<!-- 99.4% validation accuracy
Less than 20k Parameters
You can use anything from above you want. 
Less than 20 Epochs
Have used BN, Dropout, a Fully connected layer, have used GAP. 
To learn how to add different things we covered in this session, you can refer to this code: https://www.kaggle.com/enwei26/mnist-digits-pytorch-cnn-99 DONT COPY ARCHITECTURE, JUST LEARN HOW TO INTEGRATE THINGS LIKE DROPOUT, BATCHNORM, ETC. -->

# Step 2

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) # input is 1 x 28 x 28 and output is 8 x 28 x 28
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)# input is 8 x 28 x 28 and output is  16 x 28 x 28
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1) # input is 16 x 14 x 14 and output is 32 x 14 x 14
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1) # input is 32 x 14 x 14 and output is 32 x 14 x 14
        self.bn4 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.10)
        self.conv6 = nn.Conv2d(32, 10, 7) # input is 32 x 7 x 7 and output is 10 x 1 x 1
        self.bn6 = nn.BatchNorm2d(10)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # input is 10 x 7 x 7 and output is 10 x 1 x 1
        self.fc = nn.Linear(10, 10)

    def forward(self, x):
        x = self.dropout(self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x))))))))
        x = self.dropout(self.pool2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x))))))))
        x = self.bn6(self.conv6(x))
        # x = self.avgpool(x)
        x = x.view(-1,10)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)



from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


In [8]:


torch.manual_seed(1)
batch_size = 16

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [9]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    # scheduler.step()
    test(model, device, test_loader)


EPOCH: 0


Loss=0.16798463463783264 Batch_id=3749 Accuracy=95.24: 100%|██████████| 3750/3750 [00:24<00:00, 155.48it/s] 



Test set: Average loss: 0.0492, Accuracy: 9866/10000 (98.66%)

EPOCH: 1


Loss=0.06957977265119553 Batch_id=3749 Accuracy=98.09: 100%|██████████| 3750/3750 [00:23<00:00, 161.96it/s]  



Test set: Average loss: 0.0302, Accuracy: 9919/10000 (99.19%)

EPOCH: 2


...


Test set: Average loss: 0.0199, Accuracy: 9936/10000 (99.36%)

EPOCH: 18


Loss=0.02076861821115017 Batch_id=3749 Accuracy=99.43: 100%|██████████| 3750/3750 [00:23<00:00, 156.77it/s]   



Test set: Average loss: 0.0179, Accuracy: 9938/10000 (99.38%)

EPOCH: 19


Loss=0.0795389860868454 Batch_id=3749 Accuracy=99.47: 100%|██████████| 3750/3750 [00:23<00:00, 158.61it/s]    



Test set: Average loss: 0.0167, Accuracy: 9945/10000 (99.45%)

